In [1]:
import pickle
from pathlib import Path

import pandas as pd

# ------------------------------------------------------------------
# 1. chemin du pickle (adapter si besoin)
PKL = Path("C:/Users/Dell/projects/regional_flex/results/full_year.pkl")

# 2. chargement
with PKL.open("rb") as f:
    res = pickle.load(f)

# ------------------------------------------------------------------
# 3. reconstruction d’un DataFrame « long » ⇢ « large »
#    (index = pas de temps, colonnes = variables)
first_var = next(iter(res["variables"].values()))
idx = pd.date_range("2022-01-01", periods=len(first_var), freq="30min")

table = {}
for var_name, series_dict in res["variables"].items():
    # Series_dict : {t -> valeur}
    # on le remet au même index complet, les manquants ⇢ 0
    s = pd.Series(series_dict).reindex(range(len(idx)), fill_value=0.0)
    table[var_name] = s.values

df = pd.DataFrame(table, index=idx)

# ------------------------------------------------------------------
# 4. aperçu
print(df.shape)
df.head()

df.to_csv("C:/Users/Dell/projects/regional_flex/full_year.csv")


(17473, 76)


In [2]:
df.head()

,dispatch_hydro_Auvergne_Rhone_Alpes,dispatch_nuclear_Auvergne_Rhone_Alpes,dispatch_thermal_gas_Auvergne_Rhone_Alpes,dispatch_thermal_coal_Auvergne_Rhone_Alpes,dispatch_biofuel_Auvergne_Rhone_Alpes,storage_soc_STEP_Auvergne_Rhone_Alpes,storage_charge_STEP_Auvergne_Rhone_Alpes,storage_discharge_STEP_Auvergne_Rhone_Alpes,storage_soc_batteries_Auvergne_Rhone_Alpes,storage_charge_batteries_Auvergne_Rhone_Alpes,...,flow_out_Auvergne_Rhone_Alpes_Provence_Alpes_Cote_dAzur,flow_out_Nouvelle_Aquitaine_Auvergne_Rhone_Alpes,flow_out_Nouvelle_Aquitaine_Occitanie,flow_out_Nouvelle_Aquitaine_Provence_Alpes_Cote_dAzur,flow_out_Occitanie_Auvergne_Rhone_Alpes,flow_out_Occitanie_Nouvelle_Aquitaine,flow_out_Occitanie_Provence_Alpes_Cote_dAzur,flow_out_Provence_Alpes_Cote_dAzur_Auvergne_Rhone_Alpes,flow_out_Provence_Alpes_Cote_dAzur_Nouvelle_Aquitaine,flow_out_Provence_Alpes_Cote_dAzur_Occitanie
2022-01-01 00:00:00,9222.0805,2044.500,0.0,0.0,0.000,30550.0,-4.689582e-13,0.000000e+00,100.0,0.0,...,1148.3005,0.0,0.0,1462.625,0.0,0.0,201.01250,0.0,0.0,0.0
2022-01-01 00:30:00,9324.7075,2051.315,5.6,0.9,0.285,30550.0,0.000000e+00,-3.092282e-12,0.0,0.0,...,1352.8075,0.0,0.0,841.750,0.0,0.0,0.00000,0.0,0.0,0.0
2022-01-01 01:00:00,9427.3345,2044.500,0.0,0.0,0.000,30550.0,0.000000e+00,0.000000e+00,0.0,0.0,...,1629.8345,0.0,0.0,1038.525,0.0,0.0,267.01325,0.0,0.0,0.0
2022-01-01 01:30:00,9529.9615,2051.315,5.6,0.9,0.285,30550.0,0.000000e+00,0.000000e+00,0.0,0.0,...,1606.0615,0.0,0.0,733.650,0.0,0.0,279.60075,0.0,0.0,0.0
2022-01-01 02:00:00,9632.5885,2058.130,11.2,1.8,0.570,30550.0,0.000000e+00,3.092282e-12,0.0,0.0,...,1712.2885,0.0,0.0,944.775,0.0,0.0,341.18825,0.0,0.0,0.0


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Charger les fichiers
dispatch = pd.read_csv("full_year.csv", index_col=0, parse_dates=True)
prices = pd.read_csv("results/nodal_prices_full_year.csv", index_col=0, parse_dates=True)

# Créer le dossier de sortie
os.makedirs("plots_analysis", exist_ok=True)

# Ajouter les prix moyens nodaux
dispatch['price'] = prices.mean(axis=1)

# Approximer la demande comme la somme des dispatchs (toutes technos + régions)
dispatch_cols = [col for col in dispatch.columns if col.startswith('dispatch_')]
dispatch['demand'] = dispatch[dispatch_cols].sum(axis=1)

# ------------------------
# 1. Décomposition du gain par technologie
# ------------------------

gain_components = {}

techs = ['battery', 'step', 'dr', 'network']

for tech in techs:
    tech_cols = [col for col in results.columns if f"dispatch_{tech}_" in col]
    print(f"{tech}: {len(tech_cols)} colonnes trouvées.")
    if tech_cols:
        dispatch_tech = dispatch[tech_cols].sum(axis=1)
        gain_components[tech] = (dispatch_tech * dispatch['price']).sum() * 0.5

gain_df = pd.DataFrame.from_dict(gain_components, orient='index', columns=['Economic Gain (€)'])
gain_df.sort_values(by='Economic Gain (€)', ascending=False, inplace=True)

gain_df.plot(kind='bar', legend=False)
plt.title("Décomposition du gain par technologie")
plt.ylabel("Gain économique total (€)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("plots_analysis/gain_by_technology.png")
plt.close()

# ------------------------
# 2. Marge économique heure par heure
# ------------------------

# Dépense spot simulée
dispatch['spot_cost'] = dispatch['demand'] * dispatch['price'] * 0.5  # demi-horaire
# Ici on n'a pas d'objective directement => On considère que le coût optimisé est le dispatch pondéré par le prix aussi.
# C'est une approximation correcte ici.
dispatch['optimized_cost'] = dispatch['spot_cost']  # Pas d'autres coûts modélisés dans ton output
dispatch['margin'] = dispatch['spot_cost'] - dispatch['optimized_cost']

dispatch['margin'].plot()
plt.title("Marge économique heure par heure")
plt.xlabel("Temps")
plt.ylabel("Gain instantané (€)")
plt.axhline(0, color='black', linestyle='--')
plt.tight_layout()
plt.savefig("plots_analysis/margin_over_time.png")
plt.close()

# ------------------------
# 3. Prix nodaux et heures critiques
# ------------------------

plt.figure(figsize=(12,5))
dispatch['price'].plot()
plt.title("Prix nodaux (marché spot simulé)")
plt.xlabel("Temps")
plt.ylabel("Prix (€/MWh)")
plt.tight_layout()
plt.savefig("plots_analysis/nodal_prices.png")
plt.close()

# Identifier les heures critiques
threshold = dispatch['price'].quantile(0.95)
critical_hours = dispatch[dispatch['price'] > threshold]
critical_hours[['price']].to_csv("plots_analysis/critical_hours.csv")

print("✅ Analyse terminée ! Résultats disponibles dans 'plots_analysis'.")


battery: 0 colonnes trouvées.
step: 0 colonnes trouvées.
dr: 0 colonnes trouvées.
network: 0 colonnes trouvées.


C:\Users\Dell\AppData\Local\Temp\ipykernel_23248\802225327.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices = pd.read_csv("results/nodal_prices_full_year.csv", index_col=0, parse_dates=True)


TypeError: no numeric data to plot